# **Stage-1:- Creating a Knowledge Base**

- **Problem Statement**
  - Contract / defense data is stored in Excel but it’s hard to search intelligently
  - Traditional keyword search fails when the query wording is different (synonyms / rephrasing)
  - Finding the most relevant past contract descriptions takes too much manual effort
  - Even if you find a match, getting the full context (supplier, program, amount, etc.) from the correct row is difficult
  - A scalable system is needed to support quick retrieval + future AI extraction workflows

- **Proposed Solution**
  - Build a Vector Knowledge Base (KB) from the Excel dataset
  - Convert contract descriptions into semantic embeddings using a transformer model
  - Store embeddings in a FAISS vector index for fast similarity-based retrieval
  - Store all original Excel columns as metadata to return complete structured information
  - Enable searching based on meaning, not only exact words

- **Outcome**
  * Your Excel knowledge base will become **searchable by meaning (semantic search)** instead of only keywords
  * When you give a **new contract description/query**, the system will return the **most similar past contracts** instantly
  * You will be able to retrieve the **best-matching row** even if the words are different (synonyms, rephrasing, short forms)
  * Along with the match, you will also get the **complete row details** (Supplier, Program, Amount, Dates, etc.) because metadata is stored
  * Your extraction pipeline will become **more accurate**, since the LLM can be grounded with relevant historical examples
  * It will reduce **manual lookup time**, improve consistency, and make the process scalable as data grows
  * You will have a reusable **system KB (FAISS + metadata files)** that can be loaded anytime without rebuilding every time
  * This becomes a strong base for building an **agentic workflow** like: Retrieve → Validate → Extract → Store → Report




In [2]:
#!pip install -qU pandas faiss-cpu python-dateutil openpyxl langchain langgraph pydantic openai langchain-community langchain-core

In [3]:
import os
import re
import pickle
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

DEFAULT_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"


def clean_text(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def safe_to_str(x):
    if pd.isna(x):
        return ""
    return str(x).strip()


def build_system_kb_store_all_columns(
    excel_path: str,
    save_dir: str = "system_kb_store",
    model_name: str = DEFAULT_MODEL_NAME,
    batch_size: int = 64,
    embed_column: str = "Description of Contract",
):
    os.makedirs(save_dir, exist_ok=True)

    print(f"\n Loading Excel Knowledge Base: {excel_path}")
    df = pd.read_excel(excel_path)
    print(f" Loaded rows={len(df)} cols={len(df.columns)}")

    if embed_column not in df.columns:
        raise ValueError(f"Embed column '{embed_column}' not found in Excel!")

    df = df.fillna("")

    kb_texts = []
    kb_meta = []

    for idx, row in df.iterrows():
        desc = clean_text(row[embed_column])

        # Skip weak text
        if not desc or len(desc) < 20:
            continue

        meta = {"row_id": int(idx)}
        for col in df.columns:
            meta[col] = safe_to_str(row[col])

        meta[embed_column] = desc

        kb_texts.append(desc)
        kb_meta.append(meta)

    print(f"KB rows kept after cleaning: {len(kb_texts)}")

    if len(kb_texts) == 0:
        print("ERROR: No text rows remained after cleaning.")
        return None, None

    print(f"Loading embedding model: {model_name}")
    embedder = SentenceTransformer(model_name)

    print("Creating embeddings...")
    embeddings = []

    for i in range(0, len(kb_texts), batch_size):
        batch = kb_texts[i : i + batch_size]
        batch_emb = embedder.encode(
            batch, show_progress_bar=True, normalize_embeddings=True
        )
        embeddings.append(batch_emb)

    embeddings = np.vstack(embeddings).astype("float32")
    dim = embeddings.shape[1]

    print(f"Embedding shape: {embeddings.shape}")

    # Cosine similarity via Inner Product (since normalized)
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)

    index_path = os.path.join(save_dir, "system_kb.faiss")
    meta_path = os.path.join(save_dir, "system_kb_meta.pkl")

    faiss.write_index(index, index_path)
    with open(meta_path, "wb") as f:
        pickle.dump(kb_meta, f)

    print("\n System KB Created Successfully!")
    print(f"Index saved: {index_path}")
    print(f"Meta saved : {meta_path}")

    return index_path, meta_path

# Part 2: Retriever

class SystemKBRetriever:
    def __init__(self, kb_dir="system_kb_store", model_name=DEFAULT_MODEL_NAME):
        index_path = os.path.join(kb_dir, "system_kb.faiss")
        meta_path = os.path.join(kb_dir, "system_kb_meta.pkl")

        if not os.path.exists(index_path) or not os.path.exists(meta_path):
            raise FileNotFoundError("❌ KB files missing. Build KB first.")

        print(f"\n Loading FAISS index: {index_path}")
        self.index = faiss.read_index(index_path)

        print(f"Loading metadata: {meta_path}")
        with open(meta_path, "rb") as f:
            self.meta = pickle.load(f)

        print(f"Loaded KB rows: {len(self.meta)}")
        print(f"Loading embedder: {model_name}")
        self.embedder = SentenceTransformer(model_name)

    def retrieve(self, query_text: str, top_k: int = 5):
        query_text = str(query_text).strip()
        if not query_text:
            return []

        q_emb = self.embedder.encode([query_text], normalize_embeddings=True).astype(
            "float32"
        )

        scores, idxs = self.index.search(q_emb, top_k)
        results = []

        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue

            results.append({"score": float(score), "meta": self.meta[idx]})

        return results

# Run Full Pipeline (Build + Search)

if __name__ == "__main__":
    EXCEL_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\sample_data.xlsx"
    KB_DIR = "system_kb_store"

    # Step 1: Build KB
    build_system_kb_store_all_columns(
        excel_path=EXCEL_PATH,
        save_dir=KB_DIR,
        model_name=DEFAULT_MODEL_NAME,
        batch_size=64,
        embed_column="Description of Contract",
    )

    # Step 2: Load KB + Retrieve
    r = SystemKBRetriever(kb_dir=KB_DIR)

    query = "Dell Marketing L.P., Round Rock, Texas, is awarded a single-award, firm-fixed-price blanket purchase agreement"

    hits = r.retrieve(query, top_k=3)

    print("\n" + "=" * 60)
    print("TOP MATCHES")
    print("=" * 60)

    for i, h in enumerate(hits, start=1):
        print(f"\n🔹 Rank: {i}")
        print("Score:", h["score"])
        print("Supplier:", h["meta"].get("Supplier Name", ""))
        print("Market:", h["meta"].get("Market Segment", ""))
        print("System:", h["meta"].get("System Name (Specific)", ""))
        print("Row ID:", h["meta"].get("row_id", ""))


c:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



 Loading Excel Knowledge Base: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\sample_data.xlsx


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 9fbc331b-ba38-456f-a12d-677626b861e5)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


 Loaded rows=2068 cols=29
KB rows kept after cleaning: 600
Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 08bfaa31-6314-497e-90a4-e1b6b257d832)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: f1ef075f-e941-465e-b8f6-afb6a8f1e393)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-M

Creating embeddings...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


Embedding shape: (600, 384)

 System KB Created Successfully!
Index saved: system_kb_store\system_kb.faiss
Meta saved : system_kb_store\system_kb_meta.pkl

 Loading FAISS index: system_kb_store\system_kb.faiss
Loading metadata: system_kb_store\system_kb_meta.pkl
Loaded KB rows: 600
Loading embedder: sentence-transformers/all-MiniLM-L6-v2

TOP MATCHES

🔹 Rank: 1
Score: 0.6228203773498535
Supplier: Dell Inc
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)
Row ID: 379

🔹 Rank: 2
Score: 0.6228203773498535
Supplier: Dell Inc
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)
Row ID: 1

🔹 Rank: 3
Score: 0.6024371385574341
Supplier: 
Market: Unknown
System: Department of Defense Enterprise Software Initiative (DOD ESI)
Row ID: 159


# **Stage-2**

In [4]:
import os, sys
import re
import json
import pickle
import difflib
import datetime
from typing import Annotated, TypedDict, List, Dict, Any, Optional, Tuple

import pandas as pd
import faiss
from dateutil import parser
from dateutil.relativedelta import relativedelta
import getpass

# LangGraph / LangChain
from langchain_core.messages import AnyMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from openai import OpenAI

# Excel formatting
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font

sys.path.append(os.getcwd())

from prompts import (
    GEOGRAPHY_PROMPT, 
    SYSTEM_CLASSIFIER_PROMPT, 
    CONTRACT_EXTRACTOR_PROMPT, 
    VALIDATOR_FIX_PROMPT
)

In [5]:
## DEBUG LOGGING HELPERS

def log_block(title: str, content: str):
    print("\n" + "=" * 110)
    print(title)
    print("=" * 110)
    print(content)

In [6]:
## CHUNKING HELPERS (NO DATA LOSS)
def chunk_text(text: str, chunk_size: int = 1800, overlap: int = 250) -> List[str]:
    """
    Deterministic chunking: stable + safe + prevents token overflow.
    - chunk_size: number of characters per chunk
    - overlap: overlapping characters to preserve boundaries
    """
    text = str(text)
    if not text.strip():
        return []

    chunks = []
    start = 0
    n = len(text)

    while start < n:
        end = min(start + chunk_size, n)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)

        if end >= n:
            break

        start = max(0, end - overlap)

    return chunks


def pick_best_non_empty(values: List[str]) -> str:
    """Return first strong value else empty."""
    for v in values:
        if v and str(v).strip() and str(v).strip().lower() not in ["unknown", "n/a", "not applicable", "none"]:
            return str(v).strip()
    return ""

In [7]:
## RAG RETRIEVER (FAISS + METADATA)
class SystemKBRetriever:
    def __init__(self, kb_dir: str, embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.kb_dir = kb_dir
        self.embed_model = embed_model

        index_path = os.path.join(kb_dir, "system_kb.faiss")
        meta_path = os.path.join(kb_dir, "system_kb_meta.pkl")

        if not os.path.exists(index_path) or not os.path.exists(meta_path):
            raise FileNotFoundError(
                f"KB files not found in: {kb_dir}\n"
                f"Expected:\n- {index_path}\n- {meta_path}\n\n"
                f"Build KB first using your KB builder script."
            )
        print(f"Loading FAISS Index: {index_path}")
        self.index = faiss.read_index(index_path)

        print(f"Loading KB Metadata: {meta_path}")
        with open(meta_path, "rb") as f:
            self.meta = pickle.load(f)

        print(f"KB Loaded rows: {len(self.meta)}")

        self.embedder = None

    def _lazy_load_embedder(self):
        if self.embedder is None:
            from sentence_transformers import SentenceTransformer
            self.embedder = SentenceTransformer(self.embed_model)

    def retrieve(self, query_text: str, top_k: int = 3):
        import numpy as np
        query_text = str(query_text).strip()
        if not query_text:
            return []

        self._lazy_load_embedder()

        q_emb = self.embedder.encode([query_text], normalize_embeddings=True).astype("float32")
        scores, idxs = self.index.search(q_emb, top_k)

        results = []
        for score, idx in zip(scores[0], idxs[0]):
            if idx < 0:
                continue
            results.append({"score": float(score), "meta": self.meta[idx]})
        return results

In [8]:
# CONFIGURATION & FILE PATHS
TAXONOMY_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\taxonomy.json"
SUPPLIERS_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\suppliers.json"
INPUT_EXCEL_PATH = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\source_file.xlsx"
OUTPUT_CSV_PATH = "Processed_Defense_Data.csv"
RAG_KB_DIR = r"C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\system_kb_store"

In [9]:
## LLM CLIENT SETUP (OpenRouter)

if "LLMFOUNDRY_TOKEN" not in os.environ:
    os.environ["LLMFOUNDRY_TOKEN"] = getpass.getpass("Enter the LLM Foundry API Key: ")

client = OpenAI(
    api_key=f'{os.environ.get("LLMFOUNDRY_TOKEN")}:my-test-project',
    base_url="https://llmfoundry.straive.com/openai/v1/",
)

In [10]:
OPENROUTER_MODEL = "gpt-4o-mini"

# Token control per stage
MAXTOK_STAGE2 = 250
MAXTOK_STAGE3 = 850
MAXTOK_STAGE4 = 350
MAXTOK_STAGE7 = 350

retriever = SystemKBRetriever(kb_dir=RAG_KB_DIR)

Loading FAISS Index: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\system_kb_store\system_kb.faiss
Loading KB Metadata: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\system_kb_store\system_kb_meta.pkl
KB Loaded rows: 600


In [11]:
## LOAD JSON HELPERS
def load_json_file(filename, default_value):
    try:
        with open(filename, "r", encoding="utf-8") as f:
            print(f"Loaded: {filename}")
            return json.load(f)
    except Exception as e:
        print(f"Warning: Could not load {filename} ({e}). Using default.")
        return default_value


raw_taxonomy = load_json_file(TAXONOMY_PATH, {})
TAXONOMY_STR = json.dumps(raw_taxonomy, separators=(",", ":"))

SUPPLIER_LIST = load_json_file(SUPPLIERS_PATH, [
    "Dell Inc", "Boeing", "Lockheed Martin", "Raytheon Technologies",
    "Northrop Grumman", "L3Harris", "BAE Systems", "General Dynamics"
])

Loaded: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\taxonomy.json
Loaded: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\notebook\suppliers.json


In [12]:
## RULE BOOK + GEOGRAPHY

RULE_BOOK = {
    "defensive_countermeasures": {
        "triggers": ["flare", "chaff", "countermeasure", "decoy", "mju-", "ale-"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Defensive Systems', Specific: 'Defensive Aid Suite'"
    },
    "radars_and_sensors": {
        "triggers": ["radar", "sonar", "sensor", "an/apy", "an/tpy"],
        "guidance": "Market Segment: 'C4ISR Systems', System Type (General): 'Sensors'"
    },
    "ammunition": {
        "triggers": ["cartridge", "round", "projectile", " 5.56", " 7.62", "ammo"],
        "guidance": "Market Segment: 'Weapon Systems', System Type (General): 'Ammunition'"
    }
}

GEOGRAPHY_MAPPING = {
    "Sub-Saharan Africa": [
        "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde",
        "Central African Republic", "Chad", "Congo, Democratic Republic of", "Congo, Republic of",
        "Djibouti", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia",
        "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia",
        "Madagascar", "Malawi", "Mali", "Mauritius", "Mozambique", "Namibia", "Niger",
        "Nigeria", "Rwanda", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa",
        "South Sudan", "Sudan", "Tanzania", "Togo", "Uganda", "Zambia", "Zimbabwe"
    ],
    "Asia-Pacific": [
        "Australia", "Brunei", "Cambodia", "China", "Hong Kong", "Indonesia", "Japan", "Laos",
        "Malaysia", "Mongolia", "Myanmar", "New Zealand", "North Korea", "Papua New Guinea",
        "Philippines", "Singapore", "South Korea", "Taiwan", "Thailand", "Vietnam"
    ],
    "Europe": [
        "Albania", "Austria", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus",
        "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece",
        "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania", "Luxembourg",
        "Malta", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal",
        "Romania", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Turkey",
        "Ukraine", "United Kingdom"
    ],
    "Latin America": [
        "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Chile", "Colombia",
        "Costa Rica", "Cuba", "Curacao", "Dominican Republic", "Ecuador", "El Salvador", "Guatemala",
        "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Paraguay",
        "Peru", "Suriname", "Trinidad and Tobago", "Uruguay", "Venezuela"
    ],
    "Middle East and North Africa": [
        "Algeria", "Bahrain", "Egypt", "Iran", "Iraq", "Israel", "Jordan", "Kuwait", "Lebanon",
        "Libya", "Mauritania", "Morocco", "Oman", "Qatar", "Saudi Arabia", "Syria", "Tunisia",
        "United Arab Emirates", "Yemen"
    ],
    "North America": ["Canada", "USA"],
    "Russia & CIS": [
        "Armenia", "Azerbaijan", "Belarus", "Kazakhstan", "Kyrgyzstan", "Moldova", "Russia",
        "Tajikistan", "Turkmenistan", "Uzbekistan"
    ],
    "South Asia": [
        "Afghanistan", "Bangladesh", "India", "Maldives", "Nepal", "Pakistan", "Sri Lanka"
    ],
    "Unknown": [
        "Andorra", "Antigua and Barbuda", "Bhutan", "Comoros", "Dominica", "Federated States of Micronesia",
        "Fiji", "Grenada", "Kiribati", "Liechtenstein", "Marshall Islands", "Monaco", "Nauru", "Palau",
        "Palestine", "Puerto Rico", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines",
        "Samoa", "San Marino", "Sao Tom and Principe", "Solomon Islands", "Timor-Leste", "Tonga", "Tuvalu",
        "Unknown", "Vanuatu", "Vatican City", "Western Sahara"
    ]
}


In [13]:
## BASE HELPERS (Supplier + Dates + Region + Designators)

PROGRAM_TYPE_ALLOWED = [
    "Procurement",
    "Training",
    "MRO/Support",
    "RDT&E",
    "Upgrade",
    "Other Service"
]

def normalize_program_type(pt: str) -> str:
    if not pt:
        return "Other Service"

    t = str(pt).strip().lower()

    if any(k in t for k in ["mro", "support", "maintenance", "repair", "overhaul", "sustainment", "logistics"]):
        return "MRO/Support"
    if "training" in t:
        return "Training"
    if any(k in t for k in ["rdte", "research", "development", "prototype", "test and evaluation"]):
        return "RDT&E"
    if any(k in t for k in ["upgrade", "modernization", "modification"]):
        return "Upgrade"
    if any(k in t for k in ["procure", "buy", "purchase", "production", "delivery"]):
        return "Procurement"

    return "Other Service"

def get_best_supplier_match(extracted_name: str) -> str:
    if not extracted_name or str(extracted_name).strip().lower() in ["unknown", "n/a", "not applicable"]:
        return "Unknown"

    raw_name = str(extracted_name).strip()
    raw_lower = raw_name.lower()
    valid_suppliers = sorted([str(s) for s in SUPPLIER_LIST], key=len, reverse=True)

    # 1) Exact match
    for s in valid_suppliers:
        if s.lower() == raw_lower:
            return s

    # 2) Known inside extracted
    for s in valid_suppliers:
        if len(s) > 3 and s.lower() in raw_lower:
            return s

    # 3) Extracted inside known
    for s in valid_suppliers:
        if raw_lower in s.lower():
            return s

    # 4) Strict fuzzy
    matches = difflib.get_close_matches(raw_name, valid_suppliers, n=1, cutoff=0.8)
    if matches:
        return matches[0]

    return raw_name


def extract_awardee_supplier_strict(paragraph: str) -> Tuple[str, str]:
    text = str(paragraph).strip()

    patterns = [
        r"^([A-Z][A-Za-z0-9&\-\.\s]+?),\s+.*?\s+(?:is|was|has been)\s+awarded\b",
        r"^([A-Z][A-Za-z0-9&\-\.\s]+?),\s+.*?\s+received\s+an?\s+award\b",
    ]

    for pat in patterns:
        m = re.search(pat, text, flags=re.IGNORECASE)
        if m:
            raw_supplier = m.group(1).strip()
            final_supplier = get_best_supplier_match(raw_supplier)
            return final_supplier, raw_supplier

    return "Unknown", "Not Found"


def calculate_mro_months(start_date_str, end_date_text, program_type):
    if program_type != "MRO/Support":
        return "Not Applicable"
    try:
        if not start_date_str or not end_date_text:
            return "Not Applicable"

        start = pd.to_datetime(start_date_str, dayfirst=True)
        end = parser.parse(str(end_date_text), fuzzy=True)

        diff = relativedelta(end, start)
        total_months = diff.years * 12 + diff.months
        return str(max(0, int(total_months)))
    except:
        return "Not Applicable"


def get_region_for_country(country_name):
    if not country_name or str(country_name).strip().lower() in ["unknown", "n/a", "not applicable"]:
        return "Unknown"

    clean = str(country_name).strip().lower()

    if clean in ["us", "usa", "u.s.", "united states", "united states of america"]:
        return "North America"
    if clean in ["uk", "u.k.", "britain", "great britain"]:
        return "Europe"

    for region, countries in GEOGRAPHY_MAPPING.items():
        if any(c.lower() == clean for c in countries):
            return region

    return "Unknown"


DESIGNATOR_PATTERNS = [
    r"\bDDG[-\s]?\d+\b", r"\bCVN[-\s]?\d+\b", r"\bSSN[-\s]?\d+\b",
    r"\bLCS[-\s]?\d+\b", r"\bLPD[-\s]?\d+\b", r"\bLHA[-\s]?\d+\b", r"\bLHD[-\s]?\d+\b",
    r"\bF-\d+\b", r"\bB-\d+\b", r"\bC-\d+\b", r"\bA-\d+\b",
    r"\bMQ-\d+\b", r"\bRQ-\d+\b",
    r"\bAN\/[A-Z0-9\-]+\b",
    r"\b(AIM|AGM|SM|RIM|MIM)-\d+\b",
]

def extract_designators(text: str):
    text = str(text)
    found = []
    for pat in DESIGNATOR_PATTERNS:
        found.extend(re.findall(pat, text, flags=re.IGNORECASE))
    cleaned = [f.upper().replace(" ", "").replace("--", "-") for f in found]
    final, seen = [], set()
    for x in cleaned:
        if x not in seen:
            final.append(x)
            seen.add(x)
    return final


def detect_piloting_rule_based(text: str, designators: List[str]) -> str:
    t = str(text).lower()

    if any(d.startswith(("MQ-", "RQ-")) for d in designators):
        return "Uncrewed"
    if any(k in t for k in ["unmanned", "uav", "drone", "autonomous"]):
        return "Uncrewed"

    if any(d.startswith(("DDG", "CVN", "SSN", "LCS", "LPD", "LHA", "LHD")) for d in designators):
        return "Crewed"
    if "uss " in t:
        return "Crewed"

    return "Not Applicable"



In [14]:
def extract_award_value_million_strict(paragraph: str) -> str:
    """
    Extract contract AWARD value strictly from award patterns only.
    Avoids funding amounts later in the paragraph.
    Output: string in million format like '12.015'
    """
    t = str(paragraph)

    patterns = [
        r"\bis awarded a\s+\$([\d,]+(?:\.\d+)?)\b",
        r"\bare awarded an estimated aggregate ceiling of\s+\$([\d,]+(?:\.\d+)?)\b",
        r"\bare awarded an?\s+\$([\d,]+(?:\.\d+)?)\b",
        r"\bceiling of\s+\$([\d,]+(?:\.\d+)?)\b",
    ]

    for pat in patterns:
        m = re.search(pat, t, flags=re.IGNORECASE)
        if m:
            try:
                val = float(m.group(1).replace(",", ""))
                return f"{val/1_000_000:.3f}"
            except:
                pass

    return ""


def detect_quantity_should_be_na(paragraph: str) -> bool:
    """
    If paragraph contains many item quantities -> treat Quantity as Not Applicable.
    Example3: 483 missiles, 82 missiles, 156 missiles, 198 containers...
    That is NOT a single contract-level quantity.
    """
    t = str(paragraph)

    # Count numeric patterns that look like item quantities
    qty_candidates = re.findall(r"\b(\d{1,5})\b", t)

    # If too many numbers appear, it's almost always a line-item contract
    if len(qty_candidates) >= 12:
        return True

    # Specific strong markers for line-item heavy paragraphs
    if any(k in t.lower() for k in ["as follows:", "lot", "containers", "spare", "tail cap", "guidance unit"]):
        return True

    return False


def normalize_currency(cur: str) -> str:
    if not cur:
        return "USD$"
    c = str(cur).strip().upper()
    if c in ["USD", "US$", "$", "US DOLLAR", "DOLLARS"]:
        return "USD$"
    return c


In [15]:
# ==============================================================================
# STAGE 5 SPLIT ENGINE (FINAL PATCH: LINE-ITEM + OPERATOR SPLIT + WORD NUMBERS)
# ==============================================================================

def word_to_int(token: str) -> Optional[int]:
    """
    Converts word numbers to int:
    eight -> 8, twenty one -> 21

    Supports 0-99 safely.
    Returns None if not a valid word-number.
    """
    if not token:
        return None

    t = str(token).strip().lower()
    t = t.replace("-", " ")
    parts = [p for p in t.split() if p]

    ones = {
        "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
        "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10,
        "eleven": 11, "twelve": 12, "thirteen": 13, "fourteen": 14,
        "fifteen": 15, "sixteen": 16, "seventeen": 17, "eighteen": 18,
        "nineteen": 19
    }

    tens = {
        "twenty": 20, "thirty": 30, "forty": 40, "fifty": 50,
        "sixty": 60, "seventy": 70, "eighty": 80, "ninety": 90
    }

    # single word
    if len(parts) == 1:
        if parts[0] in ones:
            return ones[parts[0]]
        if parts[0] in tens:
            return tens[parts[0]]
        return None

    # two words: "twenty one"
    if len(parts) == 2:
        if parts[0] in tens and parts[1] in ones:
            return tens[parts[0]] + ones[parts[1]]
        return None

    return None


def parse_qty_token(qty_token: str) -> Optional[int]:
    """
    Converts numeric token or word-number token into int.
    """
    if qty_token is None:
        return None
    s = str(qty_token).strip().lower()

    # numeric
    if re.fullmatch(r"\d+", s):
        return int(s)

    # word-number
    return word_to_int(s)


def parse_fms_countries(paragraph: str) -> List[str]:
    """
    Extract FMS customer countries list.
    Example: 'governments of Australia, Bahrain, Belgium...'
    """
    text = str(paragraph)

    m = re.search(
        r"governments of (.+?)(?:\.\s| Work will be performed| Fiscal| This contract|$)",
        text,
        flags=re.IGNORECASE | re.DOTALL
    )
    if not m:
        return []

    block = m.group(1)
    raw = re.split(r",|\band\b", block)

    countries = []
    for c in raw:
        c = c.strip()
        if 2 < len(c) <= 40:
            countries.append(c)

    final, seen = [], set()
    for c in countries:
        if c.lower() not in seen:
            final.append(c)
            seen.add(c.lower())

    return final


def detect_multiple_supplier_award(paragraph: str) -> bool:
    """
    Detect Example2 pattern:
    Many suppliers listed with semicolons + "are awarded ..."
    """
    t = str(paragraph).strip()
    if ";" in t and re.search(r"\bare awarded\b", t, flags=re.IGNORECASE):
        if re.search(r"\([A-Z0-9]{6,}\)", t):
            return True
    return False


def parse_line_item_operator_allocations(paragraph: str) -> List[Dict[str, str]]:
    """
    ✅ Extract line-item splits like:

    '483 AIM-9X ... missiles (212 for the Navy, 187 for the Air Force and 84 for FMS customers);'
    '82 AIM-9X ... missiles (eight for the Navy, eight for the Air Force and 66 for FMS customers);'

    Output:
      [
        {"item_name": "...", "operator":"Navy", "quantity":"212", "g2g_b2g":"B2G"},
        {"item_name": "...", "operator":"Air Force", "quantity":"187", "g2g_b2g":"B2G"},
        {"item_name": "...", "operator":"Foreign Assistance", "quantity":"84", "g2g_b2g":"G2G"}
      ]
    """
    text = str(paragraph)

    # This finds: <total qty> <item desc> ( <allocation block> )
    # Example: 483 AIM-9X ... missiles (212 for the Navy, 187 for the Air Force and 84 for FMS customers)
    item_pattern = r"\b(\d+)\s+(.+?)\s*\(([^)]*for[^)]*)\)"
    matches = re.findall(item_pattern, text, flags=re.IGNORECASE)

    results = []

    for total_qty, item_desc, alloc_block in matches:
        item_desc_clean = re.sub(r"\s+", " ", item_desc).strip(" ;,.-")

        # --- BRANCH allocations (digit or word-number)
        branch_pattern = r"\b(\d+|[A-Za-z\-]+)\s+for\s+the\s+(Navy|Air Force|Army|Marine Corps|Space Force|Coast Guard)\b"
        for qty_token, op in re.findall(branch_pattern, alloc_block, flags=re.IGNORECASE):
            qty_val = parse_qty_token(qty_token)
            if qty_val is None:
                continue

            results.append({
                "item_name": item_desc_clean,
                "operator": op.title(),
                "quantity": str(qty_val),
                "g2g_b2g": "B2G"
            })

        # --- FMS allocations (digit or word-number)
        fms_pattern = r"\b(\d+|[A-Za-z\-]+)\s+for\s+(?:Foreign Military Sales\s*\(FMS\)\s*customers|Foreign Military Sales\s*customers|FMS\s*customers|FMS)\b"
        for qty_token in re.findall(fms_pattern, alloc_block, flags=re.IGNORECASE):
            qty_val = parse_qty_token(qty_token)
            if qty_val is None:
                continue

            results.append({
                "item_name": item_desc_clean,
                "operator": "Foreign Assistance",
                "quantity": str(qty_val),
                "g2g_b2g": "G2G"
            })

    return results


def split_rows_engine(base_row: dict, paragraph: str) -> List[dict]:
    """
    ✅ FINAL STAGE5 Split Engine (Your required behavior)

    SPLITS:
    1) Multi-supplier award => Supplier Name = Multiple (NO supplier split)
    2) Line-item + operator allocation split (Navy/AirForce/FMS)
    3) FMS country split ONLY for G2G rows

    ✅ Only split fields are modified, rest remain unchanged.
    """
    paragraph = str(paragraph).strip()
    row = base_row.copy()

    split_reasons = []

    # ------------------------------------------------------------------
    # 1) MULTIPLE SUPPLIER AWARD (Example2)
    # ------------------------------------------------------------------
    if detect_multiple_supplier_award(paragraph):
        row["Supplier Name"] = "Multiple"
        row["Supplier Name Evidence"] = "Multiple Suppliers (Detected)"
        row["Split Flag"] = "No"
        row["Split Reason"] = "Multiple supplier award detected (no supplier split)"
        return [row]

    rows = [row]

    # ------------------------------------------------------------------
    # 2) LINE-ITEM + OPERATOR SPLIT (Your expected output)
    # ------------------------------------------------------------------
    item_allocs = parse_line_item_operator_allocations(paragraph)

    if item_allocs:
        split_reasons.append("Line-item operator allocation split")

        new_rows = []
        for r in rows:
            for alloc in item_allocs:
                rr = r.copy()

                # ✅ Only split fields change
                rr["Customer Operator"] = alloc["operator"]
                rr["Quantity"] = alloc["quantity"]
                rr["G2G/B2G"] = alloc["g2g_b2g"]

                # ✅ Make system labels reflect line-item name (matches your expected output)
                rr["System Name (General)"] = alloc["item_name"]
                rr["System Name (Specific)"] = alloc["item_name"]

                new_rows.append(rr)

        rows = new_rows

    # ------------------------------------------------------------------
    # 3) FMS COUNTRY SPLIT (ONLY for G2G rows)
    # ------------------------------------------------------------------
    fms_countries = parse_fms_countries(paragraph)
    if fms_countries:
        split_reasons.append("FMS country split")

        final_rows = []
        for r in rows:
            is_g2g = str(r.get("G2G/B2G", "")).strip().upper() == "G2G"
            is_fms_operator = "foreign" in str(r.get("Customer Operator", "")).lower()

            if is_g2g or is_fms_operator:
                for c in fms_countries:
                    rr = r.copy()
                    rr["Customer Country"] = normalize_country_name(c)
                    rr["Customer Region"] = get_region_for_country(rr["Customer Country"])
                    final_rows.append(rr)
            else:
                final_rows.append(r)

        rows = final_rows

    # ------------------------------------------------------------------
    # Final flags
    # ------------------------------------------------------------------
    if split_reasons:
        reason = " | ".join(split_reasons)
        for r in rows:
            r["Split Flag"] = "Yes"
            r["Split Reason"] = reason
    else:
        for r in rows:
            r["Split Flag"] = "No"
            r["Split Reason"] = "No split condition found"

    return rows


In [16]:
## LLM CALL HELPER (OpenRouter Safe Wrapper)
def call_llm_json(system_prompt: str, user_prompt: str, max_tokens: int):
    """
    Safe OpenRouter call wrapper
    - JSON response enforced
    - max_tokens enforced
    """
    completion = client.chat.completions.create(
        model=OPENROUTER_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0,
        max_tokens=max_tokens,
        response_format={"type": "json_object"},
    )
    return json.loads(completion.choices[0].message.content)


In [17]:
## AGENTS / TOOLS (Chunk-wise extraction + Merge)

# Stage 1: SOURCING EXTRACTOR

class SourcingInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    url: str = Field(description="Source URL of the contract announcement/news.")
    date: str = Field(description="Contract date in Excel (string).")

@tool("sourcing_extractor")
def sourcing_extractor(paragraph: str, url: str, date: str):
    """
    Stage 1: SOURCING EXTRACTOR

    Purpose:
    - Creates the base skeleton row (stable fields).

    Output columns created:
    - Description of Contract
    - Additional Notes (Internal Only)
    - Source Link(s)
    - Contract Date
    - Reported Date (By SGA)

    Important:
    - These fields remain SAME even after splits.
    - Every split row inherits these values.
    """
    reported_date = datetime.datetime.now().strftime("%Y-%m-%d")

    notes = "Standard extraction."
    if "modification" in str(paragraph).lower():
        notes = "Contract Modification."
    if "multiple award" in str(paragraph).lower():
        notes = "Multiple award contract detected (non-supplier split)."

    return {
        "Description of Contract": paragraph,
        "Additional Notes (Internal Only)": notes,
        "Source Link(s)": url,
        "Contract Date": date,
        "Reported Date (By SGA)": reported_date
    }


In [18]:
## # Stage 2: GEOGRAPHY EXTRACTOR (Chunk -> Merge)

class GeographyInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")

@tool("geography_extractor")
def geography_extractor(paragraph: str):
    """
    Stage 2: GEOGRAPHY EXTRACTOR (Linked to prompts.py)
    """
    paragraph = str(paragraph).strip()
    if not paragraph:
        return {}

    # --- RAG Optimization (Keep your existing RAG logic here) ---
    best_hit, best_score, best_meta = rag_best_hit(paragraph, top_k=3)
    
    if best_hit and best_score >= 0.78:
        # Trust RAG if confidence is high
        kb_cust_country = safe_str(best_meta.get("Customer Country", ""))
        kb_supp_country = safe_str(best_meta.get("Supplier Country", ""))
        if kb_cust_country:
            return {
                "Customer Region": get_region_for_country(kb_cust_country),
                "Customer Country": normalize_country_name(kb_cust_country),
                "Supplier Country": normalize_country_name(kb_supp_country),
                "Domestic Content": "Indigenous" if kb_cust_country == kb_supp_country else "Imported"
            }

    # --- LLM Extraction (Using Linked Prompt) ---
    chunks = chunk_text(paragraph, chunk_size=1800, overlap=250)
    outputs = []
    
    for ch in chunks:
        try:
            # We use the imported GEOGRAPHY_PROMPT directly
            raw = call_llm_json(GEOGRAPHY_PROMPT, ch, max_tokens=250)
            outputs.append(raw)
        except Exception as e:
            continue

    # Merge Logic
    cust_country = pick_best_non_empty([o.get("Customer Country") for o in outputs]) or "Unknown"
    op = pick_best_non_empty([o.get("Customer Operator") for o in outputs]) or "Unknown"
    supp_country = pick_best_non_empty([o.get("Supplier Country") for o in outputs]) or "Unknown"
    
    cust_country = normalize_country_name(cust_country)
    supp_country = normalize_country_name(supp_country)

    return {
        "Customer Region": get_region_for_country(cust_country),
        "Customer Country": cust_country,
        "Customer Operator": op,
        "Supplier Region": get_region_for_country(supp_country),
        "Supplier Country": supp_country,
        "Domestic Content": "Indigenous" if cust_country == supp_country else "Imported"
    }

In [19]:
RAG_STRONG_THRESHOLD = 0.78   # if >= this, trust KB fully
RAG_MEDIUM_THRESHOLD = 0.70   # if >= this, use KB as strong hint

def normalize_country_name(c: str) -> str:
    if not c:
        return "Unknown"
    t = str(c).strip().lower()
    if t in ["united states", "united states of america", "us", "u.s.", "usa", "america"]:
        return "USA"
    if t in ["united kingdom", "uk", "u.k.", "britain", "great britain"]:
        return "UK"
    return str(c).strip()

def rag_best_hit(paragraph: str, top_k: int = 3):
    hits = retriever.retrieve(paragraph, top_k=top_k)
    if not hits:
        return None, 0.0, {}
    best = hits[0]
    return best, float(best.get("score", 0.0)), best.get("meta", {})

def safe_str(x, default=""):
    if x is None:
        return default
    s = str(x).strip()
    return s if s else default


In [20]:
# Stage 3: SYSTEM CLASSIFIER (Chunk -> Merge)

class SystemInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    item_focus: str = Field(description="Specific line-item focus (from Stage5), e.g. 'All up round containers'")

@tool("system_classifier")
def system_classifier(paragraph: str, item_focus: str = ""):
    """
    Stage 3: SYSTEM CLASSIFIER (Linked to prompts.py + Dynamic Taxonomy)
    """
    paragraph = str(paragraph).strip()
    
    # 1. Prepare RAG Examples (Context)
    rag_query = f"{item_focus} {paragraph}" if item_focus else paragraph
    rag_hits = retriever.retrieve(rag_query, top_k=3)
    rag_context = json.dumps([h['meta'] for h in rag_hits if h.get('score', 0) > 0.6], indent=2)

    # 2. Prepare Rule Book (Context)
    # (Simple logic to inject relevant rule hints)
    rule_hint = "No specific overrides."
    if "container" in item_focus.lower(): rule_hint = "Classify as Support Equipment / Containers."

    # 3. LINK & FORMAT THE PROMPT
    # We inject the TAXONOMY_STR variable (from your notebook config) into the prompt template
    formatted_system_prompt = SYSTEM_CLASSIFIER_PROMPT.format(
        taxonomy_reference=TAXONOMY_STR,
        rule_book_overrides=rule_hint
    )

    user_prompt = f"""
    ITEM_FOCUS: {item_focus if item_focus else "Main System"}
    TEXT: {paragraph}
    
    SIMILAR RAG EXAMPLES:
    {rag_context}
    """

    try:
        res = call_llm_json(formatted_system_prompt, user_prompt, max_tokens=850)
        
        # Post-processing (Piloting rules)
        designators = extract_designators(paragraph)
        res["System Piloting"] = detect_piloting_rule_based(paragraph, designators)
        
        return res
    except Exception as e:
        return {"Confidence": "Low", "Error": str(e)}

In [21]:
# Stage 4: CONTRACT EXTRACTOR (Chunk -> Merge + Strict Supplier)

class ContractInfoInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    contract_date: str = Field(description="Contract date as string.")

@tool("contract_extractor")
def contract_extractor(paragraph: str, contract_date: str):
    """
    Stage 4: CONTRACT EXTRACTOR (Linked to prompts.py)
    """
    paragraph = str(paragraph).strip()

    # 1. Strict Regex Supplier Search
    supplier_name, supplier_evidence = extract_awardee_supplier_strict(paragraph)

    # 2. LINK & FORMAT THE PROMPT
    # We inject the list of valid program types so the LLM follows strict rules
    formatted_contract_prompt = CONTRACT_EXTRACTOR_PROMPT.format(
        program_type_enum=str(PROGRAM_TYPE_ALLOWED)
    )

    user_prompt = f"""
    CONTRACT TEXT: {paragraph}
    DATE: {contract_date}
    """

    # 3. LLM Call
    try:
        llm_data = call_llm_json(formatted_contract_prompt, user_prompt, max_tokens=350)
    except:
        llm_data = {}

    # 4. Merge Logic (Regex vs LLM)
    program_type = normalize_program_type(llm_data.get("program_type", "Other Service"))
    
    val_formatted = extract_award_value_million_strict(paragraph) or "0.000"
    mro_months = calculate_mro_months(contract_date, llm_data.get("completion_date_text", ""), program_type)

    return {
        "Supplier Name": supplier_name,
        "Supplier Name Evidence": supplier_evidence,
        "Program Type": program_type,
        "Expected MRO Contract Duration (Months)": mro_months,
        "Quantity": "Not Applicable",
        "Value Certainty": llm_data.get("value_certainty", "Confirmed"),
        "Value (Million)": val_formatted,
        "Currency": normalize_currency(llm_data.get("currency_code", "USD")),
        "G2G/B2G": llm_data.get("g2g_b2g", "B2G"),
        "Value Note (If Any)": llm_data.get("value_note", "")
    }

In [22]:
# Stage 5: SPLITTER AGENT (FULL PARAGRAPH ALWAYS)

class SplitterInput(BaseModel):
    paragraph: str = Field(description="Full contract paragraph/description text.")
    base_row: dict = Field(description="Extracted row after Stage1-4.")

@tool("splitter_agent")
def splitter_agent(paragraph: str, base_row: dict):
  """
  Stage 5: SPLITTER AGENT
  Purpose:
    - Applies deterministic split logic to generate multiple output rows
      when paragraph has explicit multi allocations.
  Supported splits:
    - Operator/Quantity split ("212 for the Navy", "187 for the Air Force")
    - FMS country split (only for rows marked as G2G)
    - Multi-value note (does not split, only notes)

  IMPORTANT:
    - Supplier split is REMOVED to prevent wrong supplier explosions.
  """
  try:
      rows = split_rows_engine(base_row, paragraph)
      for r in rows:
          r.setdefault("Split Flag", "No")
          r.setdefault("Split Reason", "")
      return {"rows": rows}
  except Exception as e:
      base_row["Split Flag"] = "Error"
      base_row["Split Reason"] = f"Split failed: {str(e)}"
      return {"rows": [base_row]}


In [23]:
# Stage 6: QUALITY VALIDATOR AGENT

class QAInput(BaseModel):
    paragraph: str = Field(description="Original paragraph for reference.")
    rows: list = Field(description="Final split rows list output from Stage5.")

@tool("quality_validator")
def quality_validator(paragraph: str, rows: list):
    """
    Stage 6: QUALITY VALIDATOR (Hybrid: Rule-Based + KB Check)
    """
    text = str(paragraph).lower()
    validated_rows = []

    for r in rows:
        flags = []
        fixes = []

        # --- Rule Based Checks ---
        supplier = str(r.get("Supplier Name", "")).strip()
        if supplier.lower() in ["unknown", "", "multiple"]:
             # If text clearly has an award, flag it
             if "awarded" in text:
                 flags.append("Supplier Unknown")
                 fixes.append("Check LLM Fallback")

        # --- KB/Taxonomy Validation (NEW) ---
        sys_name = str(r.get("System Name (General)", "")).strip()
        
        # If we have a system name, verify it against KB context
        if sys_name and sys_name.lower() != "unknown":
            # Quick check: does this system appear in our RAG hits?
            # We assume 'retriever' is globally available as per notebook scope
            hits = retriever.retrieve(sys_name, top_k=1)
            if hits:
                top_score = hits[0]['score']
                # If score is very low, the system might be hallucinated or poorly named
                if top_score < 0.35: 
                    flags.append(f"System Name '{sys_name}' has low KB confidence ({top_score:.2f})")
                    fixes.append("Verify system name against standard taxonomy")

        # --- Final Status ---
        qa_status = "PASS" if not flags else "FAIL"
        
        rr = r.copy()
        rr["QA Status"] = qa_status
        rr["QA Flags"] = " | ".join(flags) if flags else "None"
        rr["QA Fix Suggestion"] = " | ".join(fixes) if fixes else "None"
        validated_rows.append(rr)

    return {"rows": validated_rows}

In [24]:
# Stage 7: LLM VALIDATOR (FAIL ONLY) - Chunk Aware
class LLMValidateInput(BaseModel):
    paragraph: str = Field(description="Original paragraph text.")
    row: dict = Field(description="One FAIL row to validate/correct.")

@tool("llm_fail_row_validator")
def llm_fail_row_validator(paragraph: str, row: dict):
    """
    Stage 7: VALIDATOR FIX (Linked to prompts.py)
    """
    # LINK THE PROMPT
    formatted_fix_prompt = VALIDATOR_FIX_PROMPT.format(
        failed_row_json=json.dumps(row, indent=2),
        program_type_enum=str(PROGRAM_TYPE_ALLOWED)
    )

    user_prompt = f"ORIGINAL TEXT CHUNK: {paragraph[:2000]}"

    try:
        fix = call_llm_json(formatted_fix_prompt, user_prompt, max_tokens=350)
        
        # Merge Fixes
        corrected = row.copy()
        if fix.get("Supplier Name") and fix["Supplier Name"] != "Unknown":
            corrected["Supplier Name"] = fix["Supplier Name"]
        if fix.get("Program Type"):
            corrected["Program Type"] = fix["Program Type"]
        if fix.get("Value (Million)"):
            corrected["Value (Million)"] = fix["Value (Million)"]
            
        corrected["LLM QA Fix Summary"] = fix.get("Fix Summary", "Auto-fixed by Agent")
        return {"row": corrected}

    except Exception:
        return {"row": row}

In [25]:
class AgentState(TypedDict):
    input_text: str
    input_date: str
    input_url: str

    final_data: dict
    final_rows: list
    validated_rows: list
    final_rows_post_llm: list

    messages: Annotated[List[AnyMessage], add_messages]


def stage_1_sourcing(state: AgentState):
    res = sourcing_extractor.invoke({
        "paragraph": state["input_text"],
        "url": state["input_url"],
        "date": state["input_date"],
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}


def stage_2_geography(state: AgentState):
    res = geography_extractor.invoke({"paragraph": state["input_text"]})
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}


def stage_3_system(state: AgentState):
    """
    ✅ NEW Stage3 Node:
    Runs system classification PER split row using item_focus.
    """

    paragraph = state["input_text"]
    rows = state.get("final_rows", [])

    updated_rows = []

    for r in rows:
        item_focus = str(r.get("System Name (Specific)", "")).strip()

        res = system_classifier.invoke({
            "paragraph": paragraph,
            "item_focus": item_focus
        })

        rr = r.copy()
        rr.update(res)   # merge system labels into the row
        updated_rows.append(rr)

    return {"final_rows": updated_rows}


def stage_4_contract(state: AgentState):
    res = contract_extractor.invoke({
        "paragraph": state["input_text"],
        "contract_date": state["input_date"]
    })
    new_data = state.get("final_data", {}).copy()
    new_data.update(res)
    return {"final_data": new_data}


def stage_5_split(state: AgentState):
    """
    Stage5 Node: SplitterEngine

    ✅ Ensures output is ALWAYS stored in `final_rows`
    so Stage3 SystemClassifierRAG can loop through them.
    """

    base_row = state.get("final_data", {}) or {}
    paragraph = state.get("input_text", "")

    try:
        res = splitter_agent.invoke({
            "paragraph": paragraph,
            "base_row": base_row
        })

        rows = res.get("rows", None)

        # ✅ Hard safety fallback
        if not rows or not isinstance(rows, list):
            rows = [base_row]

        return {"final_rows": rows}

    except Exception as e:
        # ✅ Never crash pipeline due to split failure
        fallback = base_row.copy()
        fallback["Split Flag"] = "Error"
        fallback["Split Reason"] = f"SplitterEngine failed: {str(e)}"
        return {"final_rows": [fallback]}


def stage_6_quality_validator(state: AgentState):
    res = quality_validator.invoke({
        "paragraph": state["input_text"],
        "rows": state["final_rows"]
    })
    return {"validated_rows": res.get("rows", state["final_rows"])}


def stage_7_llm_fix_fail_rows(state: AgentState):
    paragraph = state["input_text"]
    validated_rows = state.get("validated_rows", [])

    fixed_rows = []
    for r in validated_rows:
        if r.get("QA Status") == "FAIL":
            fix_res = llm_fail_row_validator.invoke({"paragraph": paragraph, "row": r})
            fixed_rows.append(fix_res.get("row", r))
        else:
            fixed_rows.append(r)

    return {"final_rows_post_llm": fixed_rows}

def node_system_classifier_rag(state: AgentState):
    """
    Stage3 Node (AFTER SplitterEngine)

    ✅ Runs system classification per split-row using:
       item_focus = row["System Name (Specific)"] or row["System Name (General)"]

    ✅ Updates each row with Market Segment / System Type / System Name / Evidence / Reason
    """

    paragraph = state["input_text"]
    rows = state.get("final_rows", [])

    # If split engine didn't create rows, fallback to single final_data row
    if not rows:
        base = state.get("final_data", {})
        rows = [base] if base else []

    updated_rows = []

    for r in rows:
        item_focus = str(r.get("System Name (Specific)", "")).strip()
        if not item_focus:
            item_focus = str(r.get("System Name (General)", "")).strip()

        # ✅ invoke your Stage3 tool
        sys_res = system_classifier.invoke({
            "paragraph": paragraph,
            "item_focus": item_focus
        })

        rr = r.copy()
        rr.update(sys_res)
        updated_rows.append(rr)

    return {"final_rows": updated_rows}


In [26]:
workflow = StateGraph(AgentState)

workflow.add_node("SourcingExtractor", stage_1_sourcing)
workflow.add_node("GeographyExtractor", stage_2_geography)
workflow.add_node("ContractExtractor", stage_4_contract)
workflow.add_node("SplitterEngine", stage_5_split)
workflow.add_node("SystemClassifierRAG", node_system_classifier_rag)  # ✅ only this
workflow.add_node("QualityValidator", stage_6_quality_validator)
workflow.add_node("LLMFailRowFixer", stage_7_llm_fix_fail_rows)

workflow.add_edge(START, "SourcingExtractor")
workflow.add_edge("SourcingExtractor", "GeographyExtractor")
workflow.add_edge("GeographyExtractor", "ContractExtractor")
workflow.add_edge("ContractExtractor", "SplitterEngine")   # ✅ split first
workflow.add_edge("SplitterEngine", "SystemClassifierRAG") # ✅ classify per item_focus row
workflow.add_edge("SystemClassifierRAG", "QualityValidator")
workflow.add_edge("QualityValidator", "LLMFailRowFixer")
workflow.add_edge("LLMFailRowFixer", END)
app = workflow.compile()



app = workflow.compile()

In [27]:
from IPython.display import Image, display
try:
  display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
  print(e)

Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


In [28]:
# 10) GRAPH EXPORT (OFFLINE SAFE)

def export_workflow_mermaid(app_obj, out_file="workflow.mmd"):
    mmd = app_obj.get_graph().draw_mermaid()
    with open(out_file, "w", encoding="utf-8") as f:
        f.write(mmd)
    print(f"Workflow Mermaid saved locally: {out_file}")
    return out_file

In [29]:
# 11) EXCEL HIGHLIGHTING FEATURE

def highlight_evidence_reason_columns(excel_path: str):
    wb = load_workbook(excel_path)

    # Always ensure at least one visible sheet
    ws = wb.active
    ws.sheet_state = "visible"

    header = [cell.value for cell in ws[1]]

    evidence_cols = []
    reason_cols = []

    for idx, col_name in enumerate(header, start=1):
        if isinstance(col_name, str) and "Evidence" in col_name:
            evidence_cols.append(idx)
        if isinstance(col_name, str) and "Reason" in col_name:
            reason_cols.append(idx)

    evidence_fill = PatternFill(start_color="FFF2CC", end_color="FFF2CC", fill_type="solid")
    reason_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
    header_font = Font(bold=True)

    for col_idx in evidence_cols:
        ws.cell(row=1, column=col_idx).fill = evidence_fill
        ws.cell(row=1, column=col_idx).font = header_font

    for col_idx in reason_cols:
        ws.cell(row=1, column=col_idx).fill = reason_fill
        ws.cell(row=1, column=col_idx).font = header_font

    for row in range(2, ws.max_row + 1):
        for col_idx in evidence_cols:
            ws.cell(row=row, column=col_idx).fill = evidence_fill
        for col_idx in reason_cols:
            ws.cell(row=row, column=col_idx).fill = reason_fill

    wb.save(excel_path)
    print("Evidence + Reason columns highlighted successfully.")


In [30]:
# 12) MAIN EXECUTION

if __name__ == "__main__":

    print(f"\n Loading Input File: {INPUT_EXCEL_PATH}")

    export_workflow_mermaid(app, out_file="workflow.mmd")

    try:
        df_input = pd.read_excel(INPUT_EXCEL_PATH)

        required_cols = ["Source URL", "Contract Date", "Contract Description"]
        if not all(col in df_input.columns for col in required_cols):
            raise ValueError(f"Excel file must contain columns: {required_cols}")

        print(f"🚀 Processing {len(df_input)} rows...")
        results = []

        for index, row in df_input.iterrows():
            print(f"\n🔹 Row {index + 1}/{len(df_input)}")

            desc = str(row["Contract Description"]) if pd.notna(row["Contract Description"]) else ""
            c_date = str(row["Contract Date"]) if pd.notna(row["Contract Date"]) else str(datetime.date.today())
            c_url = str(row["Source URL"]) if pd.notna(row["Source URL"]) else ""

            initial_state: AgentState = {
                "input_text": desc,
                "input_date": c_date,
                "input_url": c_url,

                "final_data": {},
                "final_rows": [],
                "validated_rows": [],
                "final_rows_post_llm": [],

                "messages": []
            }

            output_state = app.invoke(initial_state)

            rows = output_state.get("final_rows_post_llm", [])
            if not rows:
                rows = output_state.get("validated_rows", [])
            if not rows:
                rows = output_state.get("final_rows", [])
            if not rows:
                rows = [output_state.get("final_data", {})]

            results.extend(rows)

        df_final = pd.DataFrame(results)

        FINAL_COLUMNS = [
            "Customer Region", "Customer Country", "Customer Operator",
            "Supplier Region", "Supplier Country", "Domestic Content",

            "Split Flag", "Split Reason",

            "Market Segment", "Market Segment Evidence", "Market Segment Reason",
            "System Type (General)", "System Type (General) Evidence", "System Type (General) Reason",
            "System Type (Specific)", "System Type (Specific) Evidence", "System Type (Specific) Reason",
            "System Name (General)", "System Name (General) Evidence", "System Name (General) Reason",
            "System Name (Specific)", "System Name (Specific) Evidence", "System Name (Specific) Reason",
            "System Piloting", "System Piloting Evidence", "System Piloting Reason",
            "Confidence",

            "Supplier Name", "Supplier Name Evidence",
            "Program Type", "Expected MRO Contract Duration (Months)",
            "Quantity", "Value Certainty", "Value (Million)", "Currency",
            "Value (USD$ Million)", "Value Note (If Any)", "G2G/B2G",
            "Signing Month", "Signing Year",

            "QA Status", "QA Flags", "QA Fix Suggestion",
            "LLM QA Fix Summary",

            "Description of Contract",
            "Additional Notes (Internal Only)",
            "Source Link(s)",
            "Contract Date",
            "Reported Date (By SGA)"
        ]

        df_final = df_final.reindex(columns=FINAL_COLUMNS, fill_value="")
        OUTPUT_CSV_PATH = "Processed_Defense_Data.csv"
        df_final.to_csv(OUTPUT_CSV_PATH, index=False, encoding="utf-8-sig")
        print("\n Processing Complete!")
        print(f" Output CSV Saved: {OUTPUT_CSV_PATH}")
        print(df_final.head(3).to_string(index=False))

    except Exception as e:
        print(f"\n ERROR: {e}")


 Loading Input File: C:\Users\mukeshkr\Agentic-AI-Defense-Data-Extraction\data\source_file.xlsx
Workflow Mermaid saved locally: workflow.mmd
🚀 Processing 1 rows...

🔹 Row 1/1

 Processing Complete!
 Output CSV Saved: Processed_Defense_Data.csv
Customer Region Customer Country Customer Operator Supplier Region Supplier Country Domestic Content Split Flag             Split Reason Market Segment                                  Market Segment Evidence                                                                                                  Market Segment Reason System Type (General) System Type (General) Evidence                        System Type (General) Reason System Type (Specific) System Type (Specific) Evidence                                                                    System Type (Specific) Reason System Name (General) System Name (General) Evidence                                 System Name (General) Reason                                                         